In [1]:
from pyomo.environ import * 
from pyomo.opt import SolverFactory
from random import *
from math import pi,log

SEED = 7
seed(SEED)
"""
High and low values for generating coefficients for solving the problem.
Please be aware that these are treated by the RRVGG as mere suggestions.
Generally, it will not go below the low value, but it shouldn't be surprising
if it returns values above the high value. praise azathoth.
"""
low = 8
high = 20

m = ConcreteModel()

In [2]:
# Randomized random variate generator generator
def rrvgg(low=5, high=20,
          mode=None,mean=None,sd=None,mu=None,sigma=None,
          alpha=None,bet=None,lambd=None,kappa=None,meanangle=None):
    mean = mean or low + random()*(high-low)
    mode = mode or low + 0.5*(high-low)
    sd = sd or random()*low+log(low)
    mu = log(low) 
    sigma = sigma or random()  
    alpha = alpha or randint(1,low)/log(low)
    bet = bet or randint(int(0.5*low),low)/log(low)
    lambd = lambd or 1/mean
    kappa = kappa or randint(ceil(log(low)),low)
    meanangle = meanangle or mean%(pi)
    variates = [
        lambda _=0 : randint(low, high),
        lambda _=0 : uniform(low, high),
        lambda _=0 : triangular(low, high, mode),
        lambda _=0 : betavariate(alpha, bet)*(high-low)+low, 
        lambda _=0 : min(expovariate(log(high-low)*lambd)+low,high), 
        lambda _=0 : gauss(mean+log(low),sd)+low, #5
        lambda _=0 : lognormvariate(mu,sigma),
        lambda _=0 : normalvariate(mean,sd),
        lambda _=0 : vonmisesvariate(mu,kappa)*low,
        lambda _=0 : paretovariate(mu)*low,
        lambda _=0 : weibullvariate(mean, mu)+low]
    cont = variates[ randint(0,len(variates)-1) ]
    disc = lambda _=0 : ceil(cont())
    return cont, disc

cont,disc = rrvgg(low,high)

In [3]:
m.I = Set(initialize=range(1,disc()))
m.J = Set(initialize=range(1,disc()))
m.K = Set(initialize=range(1,disc()))

In [4]:
m.x1 = Var(m.I, m.K, domain = NonNegativeReals)
m.x2 = Var(m.K, m.J, domain = NonNegativeReals)
m.y  = Var(m.K, domain = Boolean)

Here we generate sets of coefficients and verify that they *should* be feasible.

In [5]:
A,Alpha,B,C1,C2,F,Q = {},{},{},{},{},{},{}
for k in m.K:
    for i in m.I:
        C1[i,k] = disc()
    for j in m.J:
        C2[k,j] = disc()
    F[k] = disc()
    Alpha[k] = disc()

for i in m.I:
    A[i] = disc()
n=0
while (sum(B.values())<sum(A.values())):
    n+=1
    for j in m.J:
        B[j] = n*disc()
n=0
while (sum(Q.values())<sum(A.values())):
    n+=1
    for k in m.K:
        Q[k] = n*disc()    
print(sum(A.values()), "  ",sum(B.values()), "  ",sum(Q.values()), "  ",)

m.a  = Param(m.I, initialize=A, domain = NonNegativeReals)
m.alpha = Param(m.K, initialize=Alpha, domain = NonNegativeReals)
m.b  = Param(m.J, initialize=B, domain = NonNegativeReals)
m.c1 = Param(m.I, m.K, initialize=C1, domain = NonNegativeReals)
m.c2 = Param(m.K, m.J, initialize=C2, domain = NonNegativeReals)
m.f  = Param(m.K, initialize=F, domain = NonNegativeReals)
m.q  = Param(m.K, initialize=Q, domain = NonNegativeReals)

316    422    499   


Parameters

In [6]:
def conservation(m,k):
    return sum(m.x1[i,k] for i in m.I) == sum(m.x2[k,j] for j in m.J)
m.conservation_constraint = Constraint(m.K, rule=conservation)

def source_out(m,i):
    return sum(m.x1[i,k] for k in m.K) == m.a[i]
m.source_out_constraint = Constraint(m.I, rule=source_out)

def process_in(m,k):
    return sum(m.x1[i,k] for i in m.I) <= m.y[k]*m.q[k]
m.process_in_constraint = Constraint(m.K, rule=process_in)

def process_out(m,j):
    return sum(m.x2[k,j] for k in m.K) <= m.b[j]
m.process_out_constraint = Constraint(m.J, rule=process_out)


Defining the objective function

In [7]:
def obj_expression(m):
    sum = 0
    for k in m.K:
        sum += m.y[k]*m.f[k]
        for i in m.I:
            sum += m.x1[i,k]*(m.c1[i,k]+m.alpha[k])
        for j in m.J:
            sum += m.x2[k,j]*m.c2[k,j]
    return sum

m.objective = Objective(rule=obj_expression, sense=minimize)

In [8]:
solver = SolverFactory('glpk')
sol = solver.solve(m, report_timing=True, tee=True)

        0.01 seconds required to write file
        0.03 seconds required for presolve
GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmpmul7wbqd.glpk.raw
 --wglp /var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmpekrpq0uy.glpk.glp
 --cpxlp /var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmp6e5vibfy.pyomo.lp
Reading problem data from '/var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmp6e5vibfy.pyomo.lp'...
/var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmp6e5vibfy.pyomo.lp:3544: warning: lower bound of variable 'x727' redefined
/var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmp6e5vibfy.pyomo.lp:3544: warning: upper bound of variable 'x727' redefined
78 rows, 749 columns, 1805 non-zeros
22 integer variables, all of which are binary
3566 lines were read
Writing problem data to '/var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmpekrpq0uy.glpk.glp'...
4166 lines were written
GLPK Integer Optimi

In [9]:
print("The solver terminated with the following decision variable values:")

K_1 = []
for k in m.K:
    if m.y[k].value == 1: K_1.append(k) 

print("\nTo Facility    ", "".join([" {:<3} ".format(str(k)) for k in K_1]))
for i in m.I: 
    #print("From Source", i,"  ".join([str(int(m.x1[i,k].value)) for k in m.K]))
    print("From Source {:>2} ".format(i),"".join([" {:<3} ".format(str(int(m.x1[i,k].value))) for k in K_1]))

print("\nFrom Facility  ", "".join([" {:<3} ".format(str(k)) for k in K_1]))
for j in m.J: 
    if sum(m.x2[k,j].value for k in K_1) > 1:
        print("To Disposal {:>2} ".format(j),"".join([" {:<3} ".format(int(m.x2[k,j].value)) for k in K_1]))

The solver terminated with the following decision variable values:

To Facility      1    2    3    4    5    6    9    12   13   14   15   17   19   20   21  
From Source  1   0    0    0    0    0    0    1    0    0    0    0    0    23   0    0   
From Source  2   0    0    0    6    0    0    0    0    0    14   0    0    0    0    0   
From Source  3   0    0    0    16   0    0    0    0    0    0    0    0    0    0    0   
From Source  4   0    0    0    0    0    0    15   0    6    0    0    0    0    0    0   
From Source  5   0    3    0    0    0    2    0    0    0    0    0    22   0    0    0   
From Source  6   18   0    0    0    0    0    0    0    0    0    0    0    0    0    0   
From Source  7   0    0    0    0    0    0    0    0    0    2    0    0    0    17   0   
From Source  8   0    0    21   0    0    0    3    0    0    0    0    0    0    0    0   
From Source  9   0    1    0    0    21   0    0    0    0    0    0    0    0    0    0   
From Source 